In [ ]:
#得到原始IDList, fileList (遍历scop-2.07-40文件夹)
#结合errorList 做差集得到 IDList
import os
from pathlib import Path

dir_path = "/home/wngys/scop/scope-2.07-40"

IDList = []

fileList = []

for child_path in os.listdir(dir_path):
    child_path = os.path.join(dir_path, child_path)
    files = sorted([os.path.join(child_path, f_path) for f_path in os.listdir(child_path) if f_path.endswith(".ent")])
    for f_path in os.listdir(child_path):
        if f_path.endswith(".ent"):
            IDList.append(f_path.split('.')[0])
            fileList.append(os.path.join(child_path, f_path))
IDset = set(IDList)
print(len(IDset))
# print(f"total_num:{total_num}")
# print(fileList[:10])

In [ ]:
# 根据蛋白质ID得到具体文件路径
idx = IDList.index("d3m2pa1")
print(fileList[idx])

In [ ]:
print(list(IDset)[:10])

In [ ]:
# 验证 AB = BA
# file_name = "valid_pair.txt"
file_name = "/home/wngys/scop/TMalign/tmalign.txt"
# file_name = "test.txt"

ABdic = {}

with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        scor1 = lines.split('\t')[2]
        scor2 = lines.split('\t')[3]
        scor3 = lines.split('\t')[4]
        scor4 = lines.split('\t')[5]
        scor5 = lines.split('\t')[6]

        if id2 + " " + id1 in ABdic:
            print(id2, id1, ABdic[id2 + " " + id1], "|",  id1, id2, scor1, scor2, scor3, scor4, scor5)
        
        ABdic[id1 + " " + id2] = scor1 + " " + scor2 + " " + scor3 + " " + scor4 + " " + scor5
# print(Hash)

In [ ]:
# GitHub 原代码 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

from Bio.PDB import PDBParser


def get_distance_matrix(pdb_path):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path).get_list()[0]
    residue_positions = get_residue_positions(structure)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
    pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
    return pdb_dist_mat


def get_residue_ids(structure):
    ids = [r.get_id()[1] for r in structure.get_residues()]
    return ids


def get_residue_positions(structure):
    residue_ids = get_residue_ids(structure)
    positions = numpy.ones((residue_ids[-1] - residue_ids[0] + 1, 3)) * float('inf')
    for residue in structure.get_residues():
        atoms = residue.get_atoms()
        for a in atoms:
            if a.get_name() == 'CA':
                positions[residue.get_id()[1] - residue_ids[0]] = a.get_coord()

    return positions

In [ ]:
# 修改的 GitHub 原代码 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

from Bio.PDB import PDBParser


def get_distance_matrix_modified(pdb_path):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path).get_list()[0]
    residue_positions = get_residue_positions(structure)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
    pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
    return pdb_dist_mat


def get_residue_ids(structure):
    ids = [r.get_id()[1] for r in structure.get_residues()]
    return ids


def get_residue_positions(structure):
    residue_ids = get_residue_ids(structure)
    positions = numpy.ones((len(residue_ids), 3)) * float('inf')
    i = 0
    for residue in structure.get_residues():
        atoms = residue.get_atoms()
        for a in atoms:
            if a.get_name() == 'CA':
                positions[i] = a.get_coord()
                i = i + 1
    return positions

In [ ]:
# # 自己解析 ent文件 生成距离矩阵 without inf（有bug）
# import sys
# import os
# import numpy
# import scipy
# import scipy.spatial

# def get_distance_matrix_without_inf(pdb_path):
#     CA_positions = []
#     with open(pdb_path, 'r') as f:
#         while True:
#             line = f.readline()
#             if not line:
#                 break
#             if line[:4] != "ATOM":
#                 continue
#             line_split = line.split()
#             if line_split[2] == "CA":
#                 position = [float(line_split[6]), float(line_split[7]), float(line_split[8])]
#                 CA_positions.append(position)
#     pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
#     return pdb_dist_mat

In [13]:
# 自己解析 ent文件 生成距离矩阵 without inf
import sys
import os
import numpy
import scipy
import scipy.spatial

def get_distance_matrix_without_inf(pdb_path):
    CA_positions = []
    with open(pdb_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line[:4] != "ATOM":
                continue
            if line[13:15] == "CA":
                position = [float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())]
                CA_positions.append(position)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
    return pdb_dist_mat

In [ ]:
# 生成14000多个距离矩阵并保存
import warnings
warnings.filterwarnings('ignore')
import numpy as np

errorList = []
errorFile = []
save_dir = "distance_matrix/"
# save_dir = ""

for file_n in fileList:
    ID = file_n.split("/")[-1].split(".")[0]
    try:
        # M = get_distance_matrix(file_n)
        # M = get_distance_matrix_modified(file_n)
        M = get_distance_matrix_without_inf(file_n)
    except:
        errorList.append(ID)
        errorFile.append(file_n)
        print("error: " + ID)
    else:
        np.save(save_dir+ID+".npy", M)
        # print("done: " + ID)

In [ ]:
print(errorList)
print(errorFile)

In [ ]:
# 测试：加载举例矩阵
M_2 = np.load("distance_matrix/d1e2aa_.npy", allow_pickle=True)
print(M_2.shape)
print(M_2)

In [ ]:
%pip install biopython

In [ ]:
# 找出所有存在inf的距离矩阵
import os

dir_path = "distance_matrix"

inf_List = []
for child_path in os.listdir(dir_path):
    child_path = os.path.join(dir_path, child_path)
    M_2 = np.load(child_path, allow_pickle=True)
    inf_n = numpy.sum(numpy.isinf(M_2))
    if inf_n != 0:
        # print(child_path, inf_n)
        inf_List.append((child_path, inf_n))

In [ ]:
print(len(inf_List))
print(inf_List[:10])

In [ ]:
dir_path = "distance_matrix/"

print(len(os.listdir(dir_path)))
print(os.listdir(dir_path)[0:10])

In [ ]:
# 矩阵对比


